# Access Token
A **valid, up-to date** access token is available to the current notebook user, so that they may use it to access additional services in the EOSC ecosystem. Let's begin with loading necessary preprequesites

In [ ]:
import requests   # HTTP Requests library
import json       # JSON parsing/emitting library
import ipywidgets # Interactive components

## 1. Read and Display the Token
We can read the token from file, and check just a few characters. Note how we never store the token into a string variable. The file gets repeatedly refreshed with a new, valid token, while the variable contents will get stale eventually.

In [ ]:
def printtoken(onlyfirst):
    print("The token starts like this: "+open('/var/run/secrets/oidc/access_token').read()[0:onlyfirst]+"... ("+str(len(open('/var/run/secrets/oidc/access_token').read())-onlyfirst)+" more characters)")
ipywidgets.interact(printtoken, onlyfirst=ipywidgets.IntSlider(description='Only first', min=16, max=1482, step=16, value=16));


## 2. Obtaining User Info
Now let's try contacting the _userinfo_ endpoint. If we get code 200, it worked. Any code from the 400 family means a problem.

In [ ]:
uir = requests.get(" https://proxy.staging.eosc-federation.eu/OIDC/userinfo", headers={"authorization": "Bearer {}".format(open('/var/run/secrets/oidc/access_token').read())}, stream=True)
print(uir)

### 2.1 Print Entitlements
From the HTTP response, let us print the content of the `entitlements` tag (only that, for brevity).

In [ ]:
print(json.dumps(json.loads(uir.content)["entitlements"], indent=2))

## 3 Listing Servers in a Hub
Another service we can access is the Interactive Notebooks Hub API. There are multiple instances available, so we may choose which one to contact, **bearing in mind that not all recognize our current identity provider**. The following drop-box gives us the choices.

The following code will retrieve the status from the selected notebooks Hub, and list the `servers` array.

In [ ]:
def hublist(hub):
    hubres = requests.get("https://"+hub+"/services/jwt/user",
                 headers={"Authorization": "Bearer {}".format(open('/var/run/secrets/oidc/access_token').read())}, stream=True)
    print(json.dumps(hubres.json()['servers'], indent=2))
ipywidgets.interact(hublist, hub=ipywidgets.Dropdown(options= ["notebooks-tst.cloud.cesnet.cz", "notebooks-stg1.cloud.cesnet.cz", "notebooks-stg2.cloud.cesnet.cz", "eu-1.notebooks.open-science-cloud-user-apps.eu", "eu-2.notebooks.open-science-cloud-user-apps.eu"], description='Endpoint:', value="notebooks-stg1.cloud.cesnet.cz"));

## 4. Conclusion
we have succesfully used the injected token to authenticate to Web-based services, to, e.g., obtain the attributes for our identity, or to check out active resources in the Interactive Notebooks Hub.